1. test.csv file does not have ground-truth (price range in test.csv file). So I split train.csv file into train and test data in 0.8:0.2 ratio. 
2. I did agglomerate cluster, kmeans++ (used from my homework 2), decisionTree learned from online course.
3. f1 score: I am not sure if the average type is micro, macro or weighted. I chose micro. 
4. Any other idea? Write-up and other method we can add-on?

Next step: Pattern summary.  



In [1]:
from platform import python_version
print("python version: ", python_version())

import pandas as pd
import numpy as np

print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)

import sklearn
print("sklearn version: ", sklearn.__version__)

# f-measure reference
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

from sklearn.metrics import f1_score

python version:  3.7.0
pandas version:  1.3.5
numpy version:  1.21.2
sklearn version:  1.0.2


In [2]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
print("test data shape: ", test_data.shape)
print("train data shape: ", train_data.shape)

test data shape:  (1000, 21)
train data shape:  (2000, 21)


Print out description of train data and test data to check validity of data. Check data to ensure that there is missing value. 

In [3]:
test_data.describe()
# issue of test_data is that it does not have price range column (ground truth column). 

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,1248.510000,0.516000,1.540900,0.517000,4.593000,0.487000,33.652000,0.517500,139.51100,...,10.054000,627.121000,1239.774000,2138.998000,11.995000,5.316000,11.085000,0.756000,0.50000,0.507000
std,288.819436,432.458227,0.499994,0.829268,0.499961,4.463325,0.500081,18.128694,0.280861,34.85155,...,6.095099,432.929699,439.670981,1088.092278,4.320607,4.240062,5.497636,0.429708,0.50025,0.500201
min,1.000000,500.000000,0.000000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.00000,...,0.000000,0.000000,501.000000,263.000000,5.000000,0.000000,2.000000,0.000000,0.00000,0.000000
25%,250.750000,895.000000,0.000000,0.700000,0.000000,1.000000,0.000000,18.000000,0.300000,109.75000,...,5.000000,263.750000,831.750000,1237.250000,8.000000,2.000000,6.750000,1.000000,0.00000,0.000000
50%,500.500000,1246.500000,1.000000,1.500000,1.000000,3.000000,0.000000,34.500000,0.500000,139.00000,...,10.000000,564.500000,1250.000000,2153.500000,12.000000,5.000000,11.000000,1.000000,0.50000,1.000000
75%,750.250000,1629.250000,1.000000,2.300000,1.000000,7.000000,1.000000,49.000000,0.800000,170.00000,...,16.000000,903.000000,1637.750000,3065.500000,16.000000,8.000000,16.000000,1.000000,1.00000,1.000000
max,1000.000000,1999.000000,1.000000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.00000,...,20.000000,1907.000000,1998.000000,3989.000000,19.000000,18.000000,20.000000,1.000000,1.00000,1.000000


Since testdata.csv does not have ground truth (price range column), we will not use testdata.csv and only use traindata.csv. For unsupervised modeling, we will not split traindata.csv. For supervised modeling, we will split traindata.csv into 0.8:0.2 ratio. 

In [4]:
train_data.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


Check if there is duplicate row in test data and train data. 

In [44]:
# Not used because test_data is not used in modeling. 
# res = True
# for i in test_data.duplicated():
#     if i:
#         print("row i:", i, "is duplicate row")
#         res = False

# if res:
#     print("There is no duplicate value in test data. ")


In [6]:
res = True
for i in train_data.duplicated():
    if i:
        print("row i:", i, "is duplicate row")
        res = False

if res:
    print("There is no duplicate value in train data. ")


There is no duplicate value in train data. 


This section: data normalization
1. z-score normalization
2. min-max normalization

z-score normalization:

In [7]:
def z_score_normalization_mobile_phone(data):
    train_data_z_score = data.copy()

    for col_name in train_data_z_score.columns:
        min = train_data_z_score[col_name].min()
        max = train_data_z_score[col_name].max()
        # print(col_name, min, max )
        std = train_data_z_score[col_name].std()
        mean = train_data_z_score[col_name].mean()
        # print(std)
        if col_name == "price_range":  # Y column and price_range column: ordinal data. will not normalize.  
            pass
        elif min == 0 and max == 1: # only normalize numerical data and does not normalize norminal data
            pass
        else:
            train_data_z_score[col_name] = (train_data_z_score[col_name] - mean)/std
    return train_data_z_score

In [8]:
train_data_z_score = z_score_normalization_mobile_phone(train_data)
print(train_data_z_score.describe())

       battery_power       blue   clock_speed     dual_sim            fc  \
count   2.000000e+03  2000.0000  2.000000e+03  2000.000000  2.000000e+03   
mean    2.084999e-16     0.4950  2.007505e-15     0.509500  1.695866e-17   
std     1.000000e+00     0.5001  1.000000e+00     0.500035  1.000000e+00   
min    -1.678398e+00     0.0000 -1.252751e+00     0.000000 -9.926421e-01   
25%    -8.801831e-01     0.0000 -1.007654e+00     0.000000 -7.623040e-01   
50%    -2.848881e-02     0.0000 -2.726702e-02     1.000000 -3.016278e-01   
75%     8.573416e-01     1.0000  8.305717e-01     1.000000  6.197247e-01   
max     1.728380e+00     1.0000  1.810959e+00     1.000000  3.383782e+00   

            four_g    int_memory         m_dep     mobile_wt       n_cores  \
count  2000.000000  2.000000e+03  2.000000e+03  2.000000e+03  2.000000e+03   
mean      0.521500 -1.363354e-16 -5.936807e-15  1.521561e-16 -1.086353e-16   
std       0.499662  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
min

min max normalization

In [9]:
def min_max_normalization_mobile_phone(data):
    train_data_min_max = data.copy()

    for col_name in train_data_min_max.columns:
        min = train_data_min_max[col_name].min()
        max = train_data_min_max[col_name].max()
        # print(col_name, min, max )
        if col_name == "price_range":
            pass
        elif min == 0 and max == 1: # only normalize numerical data and does not normalize norminal data. this if branch is not useful here. 
            pass
        else:
            train_data_min_max[col_name] = (train_data_min_max[col_name] - min)/(max-min)
    return train_data_min_max

In [10]:
train_data_min_max = min_max_normalization_mobile_phone(train_data)
print(train_data_min_max.describe())

       battery_power       blue  clock_speed     dual_sim           fc  \
count    2000.000000  2000.0000  2000.000000  2000.000000  2000.000000   
mean        0.492664     0.4950     0.408900     0.509500     0.226816   
std         0.293533     0.5001     0.326402     0.500035     0.228497   
min         0.000000     0.0000     0.000000     0.000000     0.000000   
25%         0.234302     0.0000     0.080000     0.000000     0.052632   
50%         0.484302     0.0000     0.400000     1.000000     0.157895   
75%         0.744322     1.0000     0.680000     1.000000     0.368421   
max         1.000000     1.0000     1.000000     1.000000     1.000000   

            four_g   int_memory        m_dep    mobile_wt      n_cores  ...  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000  ...   
mean      0.521500     0.484621     0.446389     0.502075     0.502929  ...   
std       0.499662     0.292673     0.320462     0.294997     0.326834  ...   
min       0.00000

In [11]:
X = train_data.drop(columns=['price_range'])
Y = train_data['price_range']
X_zscore_normal = train_data_z_score.drop(columns=['price_range'])
X_min_max_normal = train_data_min_max.drop(columns=['price_range'])

In [12]:
X_zscore_normal.describe()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
count,2.000000e+03,2000.0000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2000.000000,2000.000000,2000.000000
mean,2.084999e-16,0.4950,2.007505e-15,0.509500,1.695866e-17,0.521500,-1.363354e-16,-5.936807e-15,1.521561e-16,-1.086353e-16,8.903989e-17,1.151579e-16,6.128431e-17,-1.559863e-16,8.948398e-17,-1.582068e-18,7.938095e-17,0.761500,0.503000,0.507000
std,1.000000e+00,0.5001,1.000000e+00,0.500035,1.000000e+00,0.499662,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.426273,0.500116,0.500076
min,-1.678398e+00,0.0000,-1.252751e+00,0.000000,-9.926421e-01,0.000000,-1.655845e+00,-1.392955e+00,-1.701966e+00,-1.538790e+00,-1.635222e+00,-1.453664e+00,-1.738816e+00,-1.722281e+00,-1.734174e+00,-1.323800e+00,-1.649172e+00,0.000000,0.000000,0.000000
25%,-8.801831e-01,0.0000,-1.007654e+00,0.000000,-7.623040e-01,0.000000,-8.843135e-01,-1.046233e+00,-8.827487e-01,-6.646016e-01,-8.107264e-01,-8.165247e-01,-8.717399e-01,-8.451055e-01,-7.847870e-01,-8.647053e-01,-9.171012e-01,1.000000,0.000000,0.000000
50%,-2.848881e-02,0.0000,-2.726702e-02,1.000000,-3.016278e-01,1.000000,-2.562588e-03,-6.067634e-03,2.121490e-02,-2.275075e-01,1.376907e-02,-1.827659e-01,-1.044772e-02,2.054609e-02,-7.274678e-02,-1.760629e-01,-2.013194e-03,1.000000,1.000000,1.000000
75%,8.573416e-01,1.0000,8.305717e-01,1.000000,6.197247e-01,1.000000,8.791883e-01,1.034098e+00,8.404319e-01,1.083775e+00,8.382645e-01,6.808361e-01,8.826585e-01,8.668380e-01,8.766402e-01,7.421269e-01,9.130748e-01,1.000000,1.000000,1.000000
max,1.728380e+00,1.0000,1.810959e+00,1.000000,3.383782e+00,1.000000,1.760939e+00,1.727542e+00,1.687898e+00,1.520869e+00,1.662760e+00,2.962931e+00,1.727176e+00,1.727419e+00,1.588680e+00,2.808054e+00,1.645145e+00,1.000000,1.000000,1.000000


In [13]:
X_min_max_normal.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.492664,0.4950,0.408900,0.509500,0.226816,0.521500,0.484621,0.446389,0.502075,0.502929,0.495825,0.329137,0.501679,0.499255,0.521893,0.320389,0.500611,0.761500,0.503000,0.507000
std,0.293533,0.5001,0.326402,0.500035,0.228497,0.499662,0.292673,0.320462,0.294997,0.326834,0.303216,0.226419,0.288518,0.289880,0.300946,0.242022,0.303553,0.426273,0.500116,0.500076
min,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.234302,0.0000,0.080000,0.000000,0.052632,0.000000,0.225806,0.111111,0.241667,0.285714,0.250000,0.144260,0.250167,0.254276,0.285714,0.111111,0.222222,1.000000,0.000000,0.000000
50%,0.484302,0.0000,0.400000,1.000000,0.157895,1.000000,0.483871,0.444444,0.508333,0.428571,0.500000,0.287755,0.498665,0.505211,0.500000,0.277778,0.500000,1.000000,1.000000,1.000000
75%,0.744322,1.0000,0.680000,1.000000,0.368421,1.000000,0.741935,0.777778,0.750000,0.857143,0.750000,0.483291,0.756342,0.750534,0.785714,0.500000,0.777778,1.000000,1.000000,1.000000
max,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
Y.describe()

count    2000.000000
mean        1.500000
std         1.118314
min         0.000000
25%         0.750000
50%         1.500000
75%         2.250000
max         3.000000
Name: price_range, dtype: float64

price range: <br>
0: low cost  <br>
1: medium cost  <br>
2: high cost  <br>
3: very high cost  <br>
cluster number is 4


Agglomerative clustering: 
1. Single link
2. Complete link

In [15]:
# agglomerate cluster (n = 4)
from sklearn.cluster import AgglomerativeClustering as ac

1. Single link:
(a): without normalization
(b): z-score normalization
(c): min-max normalization

In [16]:
X_duplicate = X.copy()
Y_duplicate = Y.copy()
X_zscore_normal_duplicate = X_zscore_normal.copy()
X_min_max_normal_duplicate = X_min_max_normal.copy()

In [17]:
# single link: without normalization
prediction_single_link = ac(n_clusters=4,linkage='single').fit_predict(X_duplicate)
f1_score_single_link = f1_score(Y_duplicate, prediction_single_link, average = 'micro')
print('f1 score of single link: ', f1_score_single_link)

f1 score of single link:  0.249


In [18]:
# single link: z-score normalization
prediction_single_link = ac(n_clusters=4,linkage='single').fit_predict(X_zscore_normal_duplicate)
f1_score_single_link = f1_score(Y_duplicate, prediction_single_link, average = 'micro')
print('f1 score of single link: ', f1_score_single_link)

f1 score of single link:  0.2495


In [19]:
# single link: min-max normalization
prediction_single_link = ac(n_clusters=4,linkage='single').fit_predict(X_min_max_normal_duplicate)
f1_score_single_link = f1_score(Y_duplicate, prediction_single_link, average = 'micro')
print('f1 score of single link: ', f1_score_single_link)

f1 score of single link:  0.2505


Min max normalization and z-score normalization do not import f1 measure. 

2. Complete link

In [20]:
X_duplicate = X.copy()
Y_duplicate = Y.copy()
X_zscore_normal_duplicate = X_zscore_normal.copy()
X_min_max_normal_duplicate = X_min_max_normal.copy()

In [21]:
# complete link: without normalization
prediction_complete_link = ac(n_clusters=4,linkage='complete').fit_predict(X_duplicate)
f1_score_complete_link = f1_score(Y_duplicate, prediction_complete_link, average = 'micro')
print('f1 score of complete link: ', f1_score_complete_link)

f1 score of complete link:  0.2445


In [22]:
# complete link: z-score normalization
prediction_complete_link = ac(n_clusters=4,linkage='complete').fit_predict(X_zscore_normal_duplicate)
f1_score_complete_link = f1_score(Y_duplicate, prediction_complete_link, average = 'micro')
print('f1 score of complete link: ', f1_score_complete_link)

f1 score of complete link:  0.2385


In [23]:
# complete link: min max normalization
prediction_complete_link = ac(n_clusters=4,linkage='complete').fit_predict(X_min_max_normal_duplicate)
f1_score_complete_link = f1_score(Y_duplicate, prediction_complete_link, average = 'micro')
print('f1 score of complete link: ', f1_score_complete_link)

f1 score of complete link:  0.253


k-means++ clustering: cluster n = 4 because of price range (0 to 3)


In [36]:
from sklearn.cluster import KMeans


In [37]:
X_duplicate = X.copy()
Y_duplicate = Y.copy()
X_zscore_normal_duplicate = X_zscore_normal.copy()
X_min_max_normal_duplicate = X_min_max_normal.copy()

In [38]:
# Kmeans++: unnormalized data
predictions_kmeans = KMeans(n_clusters=4).fit_predict(X_duplicate)
f1_score_kmeans = f1_score(Y_duplicate, predictions_kmeans, average = 'micro')
print('f1 score of kmeans: ', f1_score_kmeans)

f1 score of kmeans:  0.2505


In [39]:
# Kmeans++: z-score normalization
predictions_kmeans = KMeans(n_clusters=4).fit_predict(X_zscore_normal_duplicate)
f1_score_kmeans = f1_score(Y_duplicate, predictions_kmeans, average = 'micro')
print('f1 score of kmeans: ', f1_score_kmeans)

f1 score of kmeans:  0.2355


In [40]:
# Kmeans++: min max normalization
predictions_kmeans = KMeans(n_clusters=4).fit_predict(X_min_max_normal_duplicate)
f1_score_kmeans = f1_score(Y_duplicate, predictions_kmeans, average = 'micro')
print('f1 score of kmeans: ', f1_score_kmeans)

f1 score of kmeans:  0.256


Conclusion: z-score normalization and min max normalization do not improve f1 measure of model. 

Supervised learning: DecisionTreeClassifier

Data preparation

In [29]:
train_data_copy = train_data.copy()
train_data_z_score_copy = train_data_z_score.copy()
train_data_min_max_copy = train_data_min_max.copy()

Machine learning model from codewithmosh.com. password protected. 
reference: https://codewithmosh.com/courses/enrolled/417695


(1) Unnormalized data

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

model = DecisionTreeClassifier()
X = train_data_copy.drop(columns=['price_range'])
Y = train_data_copy['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

In [31]:
# f-measure reference
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

from sklearn.metrics import f1_score
f1_score_decision_tree = f1_score(Y_test,predictions,average = 'micro')  # options include micro, marco, weighted. 
print('f1 score of decision tree: ', f1_score_decision_tree)

f1 score of decision tree:  0.8225


(2) Data using z-score normalization

In [32]:
model = DecisionTreeClassifier()
X = train_data_z_score_copy.drop(columns=['price_range'])
Y = train_data_z_score_copy['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

In [41]:
# f-measure reference
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

from sklearn.metrics import f1_score
f1_score_decision_tree = f1_score(Y_test,predictions,average = 'micro')  # options include micro, marco, weighted. 
print('f1 score of decision tree: ', f1_score_decision_tree)

f1 score of decision tree:  0.8525


(3) Data using min max normalization


In [42]:
model = DecisionTreeClassifier()
X = train_data_min_max_copy.drop(columns=['price_range'])
Y = train_data_min_max_copy['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

In [43]:

f1_score_decision_tree = f1_score(Y_test,predictions,average = 'micro')  # options include micro, marco, weighted. 
print('f1 score of decision tree: ', f1_score_decision_tree)

f1 score of decision tree:  0.835


Conclusion: z-score normalization and min max normalization slightly improves F1 measure of DecisionTreeClassifier model. 